# Week 11 Homework 11 
## Question 15.2 
In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization 
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional 
requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem 
with real data. The data is given in the file diet.xls.   

In [1]:
import pandas as pd
from pulp import *
import numpy as np

# Works Cited 
## A Blending Problem : https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html 

In [2]:
diet = pd.read_excel('diet.xls')

There are some cleaning to do as the last three rows of the data includes minimum and maximum nutrient information. Let's see how it looks first before extracting those rows. Additionally, I will store the minimum and maximum nutrient information beforehand. 

In [3]:
display(diet.tail(3))

# Collect min and max values for each nutrient
min_nut = list(diet.iloc[65, 3:len(diet.columns)])
max_nut = list(diet.iloc[66, 3:len(diet.columns)])



,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [4]:
# Removing the last three rows with min and max nutrient information
diet = diet[:-3]
display(diet.head())

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [5]:
# Creating a nested list of foods and nutrient and price info
diet_data = diet.values.tolist()

# Making a list of foods 
food = [x[0] for x in diet_data]


# Storing each information separately as dict
price = {x[0]:float(x[1]) for x in diet_data}
serving = {x[0]:x[2] for x in diet_data}
calories = {x[0]:float(x[3]) for x in diet_data}
chol = {x[0]:float(x[4]) for x in diet_data}
fat = {x[0]:float(x[5]) for x in diet_data}
sodium = {x[0]:float(x[6]) for x in diet_data}
carb = {x[0]:float(x[7]) for x in diet_data}
fiber = {x[0]:float(x[8]) for x in diet_data}
protein = {x[0]:float(x[9]) for x in diet_data}
vitA = {x[0]:float(x[10]) for x in diet_data}
vitC = {x[0]:float(x[11]) for x in diet_data}
calc = {x[0]:float(x[12]) for x in diet_data}
iron = {x[0]:float(x[13]) for x in diet_data}


## 15.2.1
Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the 
maximum and minimum daily nutrition constraints, and solve it using PuLP.  Turn in your code 
and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, 
oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)  

In [6]:
# Create Optimization Problem - we want to minimize cost
prob = LpProblem('DietProblem', LpMinimize)

In [7]:
# Create decision variables
food_vars = LpVariable.dicts("food", food, lowBound=0, cat='Continuous')
food_binary = LpVariable.dicts("food_binary", food, lowBound=0, cat='Binary')

In [8]:
# Create Objective Function
prob += lpSum([price[i] * food_vars[i] for i in food])

In [9]:
# Add Constraints - We want each good nutrient to be within min and max range
for nutrient, min_value, max_value in zip(['calories', 'chol', 'fat', 'sodium', 'carb', 
                                           'fiber', 'protein', 'vitA', 'vitC', 'calc', 'iron'], 
                                          min_nut, max_nut):
    prob += lpSum([food_vars[i] * eval(nutrient)[i] for i in food]) >= min_value, f"Min_{nutrient}_constraint"
    prob += lpSum([food_vars[i] * eval(nutrient)[i] for i in food]) <= max_value, f"Max_{nutrient}_constraint"

In [10]:
# Now let's solve the optimization problem 
soln1 = prob.solve()

sol1_dict = {}
print(LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        print( f"{v.name} = {v.varValue:.2f}")
    sol1_dict[v.name] = round(v.varValue, 2)
    
print(f"Cost: ${round(value(prob.objective),2)}")

Optimal
food_Celery,_Raw = 52.64
food_Frozen_Broccoli = 0.26
food_Lettuce,Iceberg,Raw = 63.99
food_Oranges = 2.29
food_Poached_Eggs = 0.14
food_Popcorn,Air_Popped = 13.87
Cost: $4.34


## First Optimization Solution (1 constraint ) :

The optimization solution yielded a result that the following foods : Raw Celery, Frozen Broccoli, Raw Iceberg Lettuce, Oranges, Poached Eggs, and Air Popped Popcorn gives the optimized cost of $4.34 while within the bound of minimum and maximum nutrients.

## 15.2.2

Please add to your model the following constraints (which might require adding more variables) 
and solve the new model:  
### a. 
If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will 
need two variables for each food i: whether it is chosen, and how much is part of the diet.  
You’ll also need to write a constraint to link them.)  
### b. 
Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.  
### c. 
To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be 
selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered 
meat?), just call it whatever you think is appropriate – I want you to learn how to write this 
type of constraint, but I don’t really care whether we agree on how to classify foods!]  

In [11]:
# Adding constraints 
# Given I have already made a binary food variable, that depends when it is chosen,
# I will add the following constraints to the problem.

# (a) Add constraint: If a food is selected, then a minimum of 1/10 serving must be chosen
# adding less than a large amount (1 mill) for more clarity 
for f in food:
    prob += food_vars[f] >= 0.1 * food_binary[f]
    prob += food_vars[f] <= 1000000 * food_binary[f]

    
# (b) Add constraint: At most one, but not both, of celery and frozen broccoli can be selected
prob += food_binary['Celery, Raw'] + food_binary['Frozen Broccoli'] <= 1




In [12]:
# Before I add the last constraint, I will identify protein sources that have a protein content higher than 5 mg.
prot = diet[['Foods','Protein g']][diet['Protein g'] >= 5].sort_values(by='Protein g', ascending=False)
display(prot.reset_index(drop=True))

,Foods,Protein g
0,Roasted Chicken,42.2
1,White Tuna in Water,22.7
2,Taco,20.7
3,"Malt-O-Meal,Choc",17.3
4,Pork,13.8
5,Hamburger W/Toppings,13.6
6,Vegetbeef Soup,11.2
7,Splt Pea&Hamsoup,11.1
8,Neweng Clamchwd,10.9
9,"Hotdog, Plain",10.4


In [15]:
# (C) Add constraint: at least 3 kinds of meat/poultry/fish/eggs must be selected.

# Define categories for protein sources
protein_sources = {
    'meat': ['Roasted Chicken', 'Pork', 'Bologna,Turkey', 'Frankfurter, Beef', 'Ham,Sliced,Extralean', 
             'Kielbasa,Prk', 'Hamburger W/Toppings', 'Vegetbeef Soup', 'Splt Pea&Hamsoup', 'Hotdog, Plain', 'Taco'],
    'poultry': ['Roasted Chicken', 'Bologna,Turkey'],
    'fish': ['Sardines in Oil', 'White Tuna in Water'],
    'eggs': ['Poached Eggs', 'Scrambled Eggs']
}

# Collect all protein source foods
all_protein_foods = [food for foods in protein_sources.values() for food in foods]

# Add constraint: At least 3 kinds of meat/poultry/fish/eggs must be selected
prob += lpSum(food_binary[food] for food in all_protein_foods) >= 3


In [16]:
soln2 = prob.solve()

sol2_dict = {}
print(LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        print( f"{v.name} = {v.varValue:.2f}")
    sol2_dict[v.name] = round(v.varValue, 2)
    
print(f"Cost: ${round(value(prob.objective),2)}")

Optimal
food_Bologna,Turkey = 0.10
food_Celery,_Raw = 42.60
food_Lettuce,Iceberg,Raw = 82.09
food_Oranges = 3.09
food_Peanut_Butter = 2.01
food_Poached_Eggs = 0.13
food_Popcorn,Air_Popped = 13.20
food_binary_Bologna,Turkey = 1.00
food_binary_Celery,_Raw = 1.00
food_binary_Lettuce,Iceberg,Raw = 1.00
food_binary_Oranges = 1.00
food_binary_Peanut_Butter = 1.00
food_binary_Poached_Eggs = 1.00
food_binary_Popcorn,Air_Popped = 1.00
Cost: $4.5


## Second Optimization Solution (4 constraints ) :

The optimization solution that had a total of 4 constraints (1 nutrient + 3 other constraints) yielded a result that the following foods : Turkey Bologna, Raw Celery, Peanut Butter, Frozen Broccoli, Raw Iceberg Lettuce, Oranges, Poached Eggs, and Air Popped Popcorn gives the optimized cost of $4.50 while within the bound of minimum and maximum nutrients.